In [1]:
import logging
import sys
sys.path.append('/remote/gpu02/crescenzo/MadMiner')

In [2]:
# MadMiner output
logging.basicConfig(
    format="%(asctime)-5.5s %(name)-20.20s %(levelname)-7.7s %(message)s",
    datefmt="%H:%M",
    level=logging.INFO,
)

# Output of all other modules (e.g. matplotlib)
for key in logging.Logger.manager.loggerDict:
    if "madminer" not in key:
        logging.getLogger(key).setLevel(logging.WARNING)

In [7]:
from madminer import MadMiner
import numpy as np

In [8]:
miner = MadMiner()

miner.add_parameter(
    lha_block="dim6",
    lha_id=2,
    parameter_name="CWL2",
    morphing_max_power=2,
)
miner.add_parameter(
    lha_block="dim6",
    lha_id=5,
    parameter_name="CPWL2",
    morphing_max_power=2,
)

11:43 madminer.core.madmin INFO    Adding parameter: CWL2 (LHA: dim6 2, Power: 2, Range: (0.0, 1.0))
11:43 madminer.core.madmin WARNING Resetting benchmarks and morphing
11:43 madminer.core.madmin INFO    Adding parameter: CPWL2 (LHA: dim6 5, Power: 2, Range: (0.0, 1.0))
11:43 madminer.core.madmin WARNING Resetting benchmarks and morphing


In [9]:
miner.cs_set_morphing(max_overall_power=2)

11:43 madminer.core.madmin INFO    Optimizing basis for morphing the cross sections only
11:43 madminer.core.madmin INFO    Testing out


In [ ]:
mg_dir = "/remote/gpu02/crescenzo/MG5_aMC_v3_5_3"
for j,elem in miner.morpher.cs_basis:
    miner.run(
    sample_benchmark=elem,
    mg_directory=mg_dir,
    mg_process_directory=f"./mg_processes/signal_{j}",
    proc_card_file="cards/proc_card_signal.dat",
    param_card_template_file="cards/param_card_template.dat",
    run_card_file="cards/run_card_signal_large.dat",
    log_directory=f"logs/signal_{j}",
    python_executable="python3",
    only_prepare_script=True
)

Now run the MadGraph simulation. Compute externally the cross sections and set them as reduced cross sections

In [6]:
##JUST EXAMPLE!
##SUPPOSE I HAVE MAX POWER = 2 -> 6 BENCHMARK POINTS
cs_from_madgraph = np.linspace(0.001,0.010,6)
reduced_cs = cs_from_madgraph[1:cs_from_madgraph.size]/cs_from_madgraph[0]
##NOTICE THE NEW SIZE IS N_BENCHMARKS - 1

miner.morpher.set_reduced(reduced_cs)
miner.save("save_file.h5")